In [1]:
import pandas as pd

In [2]:
# World Cup Matches
matches = pd.read_csv("../WorldCupMatches.csv")

# World Cup Players
players = pd.read_csv("../WorldCupPlayers.csv")

# World Cups
cups = pd.read_csv("../WorldCups.csv")

In [3]:
matches.head()

,Year,Datetime,Stage,Home Team Goals,Away Team Goals,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,2010.0,11 Jun 2010 - 16:00,Group A,1.0,1.0,84490.0,0.0,0.0,Ravshan IRMATOV (UZB),ILYASOV Rafael (UZB),KOCHKAROV Bakhadyr (KGZ),249722.0,300061454.0,RSA,MEX
1,2010.0,11 Jun 2010 - 20:30,Group A,0.0,0.0,64100.0,0.0,0.0,NISHIMURA Yuichi (JPN),SAGARA Toru (JPN),JEONG Hae Sang (KOR),249722.0,300061453.0,URU,FRA
2,2010.0,12 Jun 2010 - 13:30,Group B,2.0,0.0,31513.0,1.0,0.0,HESTER Michael (NZL),HINTZ Jan Hendrik (NZL),MAKASINI Tevita (TGA),249722.0,300061459.0,KOR,GRE
3,2010.0,12 Jun 2010 - 16:00,Group B,1.0,0.0,55686.0,1.0,0.0,Wolfgang STARK (GER),SALVER Jan-Hendrik (GER),PICKEL Mike (GER),249722.0,300061460.0,ARG,NGA
4,2010.0,12 Jun 2010 - 20:30,Group C,1.0,1.0,38646.0,1.0,1.0,SIMON Carlos (BRA),HAUSMANN Altemir (BRA),BRAATZ Roberto (BRA),249722.0,300061466.0,ENG,USA


In [4]:
players.head()

,RoundID,MatchID,Team Initials,Coach Name,Player Name
0,201,1090,USA,MILLAR Bob (USA),Tom FLORIE
1,201,1093,BRA,DE CARVALHO Pindaro (BRA),PREGUINHO
2,201,1098,PER,BRU Francisco (ESP),Placido GALINDO
3,201,1091,BRA,DE CARVALHO Pindaro (BRA),PREGUINHO
4,201,1089,PAR,DURAND LAGUNA Jose (ARG),Luis VARGAS PENA


# Add new columns for players

In [39]:
cows = matches.columns
cows = list(cows)

In [40]:
new = [
    "Home Player1", "Home Player2", 
    "Home Player3", "Home Player4", 
    "Home Player5", "Home Player6", 
    "Home Player7", "vPlayer8", 
    "Home Player9","Home Player10",
    "Home Player11","Home Coach Name",
    "Away Player1", "Away Player2", 
    "Away Player3", "Away Player4", 
    "Away Player5", "Away Player6", 
    "Away Player7", "Away Player8", 
    "Away Player9","Away Player10",
    "Away Player11","Away Coach Name",
]

cows += new
#cows

In [41]:
final = pd.DataFrame(columns=cows)

In [48]:
for i in range(len(matches)):
    features = matches.iloc[i].values
    ID = matches.iloc[i]["MatchID"]
    HomeInitials = matches.iloc[i]["Home Team Initials"]
    
    #print(ID)
    for i in range(len(players)):
        IDPlayers = players.iloc[i]["MatchID"]
        if ID == IDPlayers:
            player = players.iloc[i]["Player Name"]
            if HomeInitials == players.iloc[i]["Team Initials"]:
                pass
            else:
                pass
            
    
    
pd.Series()
    #print(features)
#new_matches = new_matches.append(matches.iloc[i])

KeyboardInterrupt: 

In [61]:
a = pd.Series([1, 2, 3], index=[1,2,3])

In [63]:
a.append([4], index=[4])

TypeError: append() got an unexpected keyword argument 'index'